# 因子回测

## 导入模块

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy import stats
import feather
import os
import statsmodels.api as sm
import sunlandsdatasdk as sd

## 定义回测区间

In [2]:
start_date = '2019-01-01'
end_date = '2025-01-01'

## 读入和处理数据

### 读入 ST 股票

读入全公司列表

In [3]:
issues = feather.read_dataframe('../data/issues_daily.feather')
issues_tot = issues['issue'].sort_values().unique()

读入 ST 股票

In [4]:
# sd.auth('*', '*')
# st = sd.get_st_issue(issues=list(issues_tot), start_date=start_date, end_date=end_date)
# st = st.set_index('date').stack().reset_index().rename(columns={'level_1': 'issue', 0: 'st'})
# feather.write_dataframe(st, '../data/st.feather')
st = feather.read_dataframe('../data/st.feather')

### 读入日线数据

读入日线数据

In [5]:
price_1d = feather.read_dataframe('../data/StockPriceK1d_20241231.feather')

计算调仓日

In [6]:
trade_date = price_1d['date'].sort_values().unique()
adj_date = trade_date[(trade_date >= start_date) & (trade_date <= end_date)]

计算每两个调仓日之间的收益

In [7]:
price_adj = price_1d.loc[(price_1d['date'] >= start_date) & (price_1d['date'] <= end_date), ['issue', 'date', 'ret']].copy()
price_adj['ret_next'] = price_adj.groupby('issue')['ret'].shift(-1)
price_adj = price_adj.dropna(subset='ret_next')

剔除股票

In [8]:
def newly_listed(date, timedelta='180D'):
    start_date = date.min()
    newly = (date <= start_date + pd.Timedelta(timedelta))
    return newly

price_1d['newly_listed'] = (
    price_1d
        .groupby('issue')['date']
        .transform(newly_listed)
)

price_adj = pd.merge(
    price_adj,
    price_1d[['issue', 'date', 'newly_listed', 'is_limit_sell', 'is_limit_buy']],
    on=['issue', 'date'],
    how='left'
)
price_adj = pd.merge(
    price_adj,
    st,
    on=['issue', 'date'],
    how='left'
)
price_adj['st'] = price_adj['st'].ffill()
price_adj = price_adj[
    (~price_adj['newly_listed']) &
    (~price_adj['is_limit_buy'].astype(bool)) &
    (~price_adj['is_limit_sell'].astype(bool))
]
price_adj = price_adj[~price_adj['st']]
price_adj = price_adj[['issue', 'date', 'ret', 'ret_next']]

C:\Users\admin\AppData\Local\Temp\ipykernel_3804\2002057231.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  price_adj['st'] = price_adj['st'].ffill()


### 计算基准

中证 500

In [9]:
# sd.auth('*', '*')
# index_bar = sd.get_index_bar('000906', start_date='2018-12-28', end_date=end_date)
# index_bar['date'] = pd.to_datetime(index_bar['date'])
# index_bar = index_bar.drop(columns='_id')

# index_bar['ret'] = index_bar['close'] / index_bar['close'].shift(1) - 1
# index_bar['ret_next'] = index_bar['ret'].shift(-1)
# index_bar = index_bar.dropna()
# index_bar = index_bar.set_index('date')
# ret_index = index_bar[['ret', 'ret_next']]

# os.makedirs('../data/benchmark/', exist_ok=True)
# feather.write_dataframe(ret_index, '../data/benchmark/benchmark_000906_daily.feather')
# ret_index

全样本等权

In [10]:
# ret_sample = (
#     price_adj
#         .groupby('date')[['ret', 'ret_next']].mean()
# )
# os.makedirs('../data/benchmark/', exist_ok=True)
# feather.write_dataframe(ret_sample, '../data/benchmark/benchmark_sample_daily.feather')
# ret_sample

## IC 测试

In [11]:
def IC_calc_once(factor, factor_col, ret_col):
    IC, p = stats.spearmanr(factor[factor_col], factor[ret_col])
    return IC

def IC_calc(factor, factor_col, prc):
    f = factor.copy()
    f = pd.merge(
        f,
        prc,
        on=['issue', 'date'],
        how='inner'
    )  
    IC = (
        f
            .groupby('date')[[factor_col, 'ret_next']]
            .apply(IC_calc_once, factor_col=factor_col, ret_col='ret_next')
    )
    return IC

## 分组测试

### 计算分组收益和换手率

In [12]:
def grouped_ret(f, factor_col):
    f_date = f.copy()
    qcut = pd.qcut(
        f_date[factor_col],
        q=10,
        labels=np.arange(1, 11)
    )
    f_date['group'] = qcut
    ret = f_date.groupby('group', observed=False)['ret_next'].mean()
    return ret

def top_weight(f, factor_col):
    f_date = f.copy()
    f_date['group'] = pd.qcut(
        f_date[factor_col],
        q=10,
        labels=np.arange(1, 11)
    )
    top = pd.Series(0., index=issues_tot)
    issues_top = f_date.loc[f_date['group'] == 10, 'issue'].to_list()
    top[issues_top] = 1 / len(issues_top)
    return top

def grouped_calc(factor, factor_col, prc, factor_name=None):
    if factor_name is None:
        factor_name = factor_col
    f = pd.merge(
        factor,
        prc,
        on=['issue', 'date'],
        how='inner'
    )
    ret = (
        f
            .groupby('date')[[factor_col, 'ret', 'ret_next']]
            .apply(grouped_ret, factor_col=factor_col)
    )
    top = (
        f
            .groupby('date')[[factor_col, 'issue']]
            .apply(top_weight, factor_col=factor_col)
    )
    turnover = np.abs(top - top.shift(1)).sum(axis=1)
    return ret, turnover

### 画分组收益图和多空收益曲线

In [13]:
def grouped_plot(ret, ret_excess, factor_name, dirname=None):
    ret_annual = (1 + ret.mean()) ** 252 - 1
    plt.bar(ret_annual.index, ret_annual, color='#a60021')
    plt.xlabel('Group')
    plt.ylabel('Annualized Return')
    plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=0))
    plt.title(factor_name + ' Annualized Return')
    if dirname is None:
        plt.show()
    else:
        os.makedirs(dirname, exist_ok=True)
        plt.savefig(dirname + '/grouped_' + factor_name + '.png')
        plt.close()
    
    long = (1 + ret[10]).cumprod()
    long_excess = (1 + ret_excess[10]).cumprod()
    long_short = (1 + ret[10] - ret[1]).cumprod()

    fig, ax1 = plt.subplots()
    ax1.plot(long, linewidth=3, color='#a60021', label='long')
    ylim = ax1.get_ylim()
    ax1.fill_between(
        long_excess.index, 0, long_excess,
        color='#a5a5a5', label='long_excess',
    )
    ax1.set_ylim(ylim)
    ax1.set_ylabel('Net Value (long)')
    ax1.grid()
    
    ax2 = ax1.twinx()
    ax2.plot(long_short, linewidth=3, color='#ffc000', label='long-short')
    ax2.set_ylabel('Net Value (long-short)')
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2)
    plt.xlabel('Time')
    plt.title(factor_name + ' Grouping Test')
    if dirname is None:
        plt.show()
    else:
        os.makedirs(dirname, exist_ok=True)
        plt.savefig(dirname + '/nv_' + factor_name + '.png')
        plt.close()

### 计算最大回撤

In [14]:
def max_drawdown_calc(ret):
    cumret = (1 + ret).cumprod()
    running_max = cumret.cummax()
    drawdown = cumret / running_max - 1
    return drawdown.min()

### 一个收益率曲线的统计量

In [15]:
def ret_stats_calc(ret, factor_name, group_name):
    df_grouped = pd.DataFrame(columns=[
        'factor_name', 'group_name',
        'tot', 'annual', 'volatility',
        'max_drawdown', 'sharpe', 'calmar'
    ])
    tot = (1 + ret).prod() - 1
    annual = (1 + ret.mean()) ** 252 - 1
    volatility = ret.std() * np.sqrt(252)
    max_drawdown = max_drawdown_calc(ret)
    sharpe = annual / volatility
    calmar = annual / max_drawdown
    df_grouped.loc[0] = [
        factor_name, group_name,
        tot, annual, volatility,
        max_drawdown, sharpe, calmar
    ]
    return df_grouped

def grouped_stats_calc(long, excess, long_short, turnover, factor_name):
    df_grouped = pd.DataFrame(columns=[
        'factor_name',
        'long_annual', 'excess_annual', 'excess_sharpe',
        'long_short_annual',
        'max_drawdown', 'turnover_annual'
    ])

    long_annual = (1 + long.mean()) ** 252 - 1
    excess_annual = (1 + excess.mean()) ** 252 - 1
    std_annual = np.sqrt(252) * excess.std()
    excess_sharpe = excess_annual / std_annual
    long_short_annual = (1 + long_short).mean() ** 252 - 1
    max_drawdown = max_drawdown_calc(long_short)
    turnover_annual = turnover.mean() * 252
    df_grouped.loc[0]=[
        factor_name,
        long_annual, excess_annual, excess_sharpe,
        long_short_annual,
        max_drawdown, turnover_annual
    ]
    return df_grouped

def grouped_stats_calc_2(long, excess, long_short, turnover, factor_name):
    df_long = ret_stats_calc(long, factor_name, 'long')
    df_excess = ret_stats_calc(excess, factor_name, 'excess')
    df_long_short = ret_stats_calc(long_short, factor_name, 'long_short')
    df_grouped = pd.concat([df_long, df_excess, df_long_short], axis=0)
    turnover_annual = turnover.mean() * 252
    df_grouped['turnover_annual'] = turnover_annual
    return df_grouped

## 一键回测

In [16]:
def backtest(filename, factor_col, factor_name=None, benchmark=None, sample=None, dirname=None):
    if factor_name is None:
        factor_name = factor_col
    factor = feather.read_dataframe(filename)

    if sample is None:
        prc = price_adj
    else:
        map_sample_file = {
            'hs800': '../data/indices/hs800_20241231.feather',
            'hs1000': '../data/indices/hs1000_20241231.feather'
        }
        sample_file = map_sample_file[sample]
        df_sample = feather.read_dataframe(sample_file)
        prc = pd.merge(
            price_adj,
            df_sample,
            on=['date', 'issue'],
            how='inner'
        )

    df_IC = pd.DataFrame(columns=['factor', 'IC_mean', 'winning', 'IC_IR', 't', 'max', 'min'])
    IC = IC_calc(factor, factor_col=factor_col, prc=prc)

    sign = np.sign(IC.mean())
    factor[factor_col] *= sign

    winning = (np.sign(IC) == sign).sum() / len(IC)
    df_IC.loc[0] = [
        factor_name,
        IC.mean(),
        winning,
        IC.mean() / IC.std(),
        np.sqrt(len(IC)) * IC.mean() / IC.std(),
        IC.max(),
        IC.min()
    ]

    df_grouped = pd.DataFrame(columns=[
        'factor',
        'long_annual', 'excess_annual', 'long_sharpe',
        'long_short',
        'max_drawdown', 'turnover_annual'
    ])
    grouped, turnover = grouped_calc(
        factor, factor_col=factor_col,
        factor_name=factor_name, prc=prc
    )
    if benchmark != None:
        ret_benchmark = feather.read_dataframe(benchmark)
    ret_benchmark = ret_benchmark.reindex(grouped.index)
    excess = grouped.sub(ret_benchmark['ret_next'], axis=0)

    grouped_plot(grouped, excess, factor_name, dirname)
    df_grouped = grouped_stats_calc_2(
        grouped[10], excess[10], grouped[10] - grouped[1], turnover,
        factor_name=factor_name
    )
    
    return df_IC, df_grouped

### 一组因子回测

In [17]:
def set_backtest(filenames, factor_cols, factor_names, benchmark, sample=None, dirname=None):
    if factor_names is None:
        factor_names = factor_cols
    IC = None
    grouped = None
    for filename, factor_col, factor_name in zip(filenames, factor_cols, factor_names):
        IC_factor, grouped_factor = backtest(
            filename=filename,
            factor_col=factor_col, factor_name=factor_name,
            benchmark=benchmark, sample=sample,
            dirname=dirname
        )
        IC = pd.concat([IC, IC_factor])
        grouped = pd.concat([grouped, grouped_factor])
    display(IC)
    display(grouped)

## 跳跃收益因子回测

In [18]:
IC, grouped = backtest(
    filename='../data/ret_jump_daily/neutral_ret_jump.feather',
    factor_col='neutral_factor', factor_name='ret_jump_neutral',
    benchmark='../data/benchmark/benchmark_sample_daily.feather',
    dirname='../data/ret_jump_daily/backtest1/'
)
display(IC)
display(grouped)

,factor,IC_mean,winning,IC_IR,t,max,min
0,ret_jump_neutral,-0.037254,0.766768,-0.68972,-24.982706,0.252839,-0.293245


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,ret_jump_neutral,long,0.664489,0.139347,0.254611,-0.376512,0.547292,-0.370099,104.098469
0,ret_jump_neutral,excess,0.183450,0.034201,0.050542,-0.098921,0.676676,-0.345739,104.098469
0,ret_jump_neutral,long_short,6.980468,0.496567,0.092263,-0.094026,5.382082,-5.281156,104.098469


## 点度中心性回测

### N_connect_0_1

In [19]:
cores = np.array(['_num', '_size', '', '_residual'])
filenames = '../data/N_connect_daily/N_connect_0_1/neutral_N_connect' + cores + '.feather'
factor_cols = ['neutral_factor'] * len(cores)
factor_names = 'N_connect' + cores
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    dirname='../data/N_connect_daily/N_connect_0_1/backtest1/'
)

print('-' * 10 + ' hs800 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs800',
    dirname='../data/N_connect_daily/N_connect_0_1/backtest1_hs800/'
)

print('-' * 10 + ' hs1000 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs1000',
    dirname='../data/N_connect_daily/N_connect_0_1/backtest1_hs1000/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,0.016868,0.642314,0.366764,13.294863,0.210242,-0.144425
0,N_connect_size,0.018185,0.661339,0.395516,14.337113,0.191593,-0.183746
0,N_connect,0.018942,0.652968,0.391465,14.190280,0.203052,-0.163709
0,N_connect_residual,0.006060,0.568493,0.137983,5.001769,0.174284,-0.152291


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.798237,0.146104,0.217889,-0.288697,0.670545,-0.506082,114.734050
0,N_connect_num,excess,0.654602,0.111731,0.136531,-0.258895,0.818359,-0.431569,114.734050
0,N_connect_num,long_short,0.648626,0.102571,0.059490,-0.076128,1.724164,-1.347345,114.734050
0,N_connect_size,long,0.862028,0.155113,0.223126,-0.277034,0.695180,-0.559904,34.535057
0,N_connect_size,excess,0.722316,0.120470,0.137507,-0.269707,0.876103,-0.446671,34.535057
0,N_connect_size,long_short,0.907130,0.133815,0.059616,-0.089692,2.244625,-1.491942,34.535057
0,N_connect,long,0.855766,0.153550,0.219897,-0.278193,0.698284,-0.551956,70.336894
0,N_connect,excess,0.710956,0.118955,0.136910,-0.261957,0.868855,-0.454102,70.336894
0,N_connect,long_short,0.934248,0.137107,0.062766,-0.084585,2.184420,-1.620928,70.336894
0,N_connect_residual,long,0.266277,0.072084,0.220183,-0.360733,0.327380,-0.199825,75.956313


---------- hs800 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,0.008288,0.544140,0.111279,4.033757,0.252542,-0.252497
0,N_connect_size,0.008593,0.543379,0.117421,4.256393,0.273842,-0.207764
0,N_connect,0.009250,0.548706,0.119740,4.340468,0.273072,-0.219237
0,N_connect_residual,0.002738,0.528158,0.040065,1.452307,0.260301,-0.215565


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.215818,0.056787,0.188408,-0.281318,0.301405,-0.201861,105.307052
0,N_connect_num,excess,0.112827,0.025083,0.092522,-0.210308,0.271100,-0.119267,105.307052
0,N_connect_num,long_short,-0.070053,-0.010151,0.086399,-0.241319,-0.117485,0.042063,105.307052
0,N_connect_size,long,0.318512,0.074365,0.193355,-0.270784,0.384602,-0.274627,33.365893
0,N_connect_size,excess,0.214940,0.042135,0.088769,-0.207844,0.474661,-0.202724,33.365893
0,N_connect_size,long_short,0.071746,0.017339,0.088424,-0.150183,0.196092,-0.115454,33.365893
0,N_connect,long,0.275847,0.067093,0.190833,-0.289525,0.351580,-0.231735,67.360639
0,N_connect,excess,0.171451,0.035081,0.090992,-0.180555,0.385537,-0.194295,67.360639
0,N_connect,long_short,0.110491,0.024604,0.091813,-0.190017,0.267980,-0.129483,67.360639
0,N_connect_residual,long,0.083168,0.034081,0.190733,-0.391783,0.178685,-0.086990,72.872992


---------- hs1000 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,0.015001,0.593607,0.257916,9.349223,0.355341,-0.177800
0,N_connect_size,0.017183,0.627093,0.294989,10.693090,0.212327,-0.193896
0,N_connect,0.017498,0.608067,0.289251,10.485116,0.252842,-0.196789
0,N_connect_residual,0.007135,0.557839,0.122732,4.448932,0.228120,-0.212585


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.710677,0.136747,0.224262,-0.309418,0.609764,-0.441950,117.084639
0,N_connect_num,excess,0.589038,0.102654,0.133398,-0.258585,0.769529,-0.396981,117.084639
0,N_connect_num,long_short,0.816369,0.125253,0.084393,-0.079417,1.484168,-1.577162,117.084639
0,N_connect_size,long,0.578095,0.120544,0.229296,-0.321972,0.525712,-0.374392,35.967198
0,N_connect_size,excess,0.473720,0.086935,0.134014,-0.277650,0.648695,-0.313109,35.967198
0,N_connect_size,long_short,0.667395,0.106567,0.080216,-0.086526,1.328502,-1.231618,35.967198
0,N_connect,long,0.675596,0.132910,0.226959,-0.327560,0.585613,-0.405759,73.982544
0,N_connect,excess,0.559816,0.098932,0.134701,-0.258560,0.734451,-0.382625,73.982544
0,N_connect,long_short,0.877175,0.132331,0.083766,-0.096000,1.579761,-1.378448,73.982544
0,N_connect_residual,long,0.238736,0.069044,0.226586,-0.414882,0.304714,-0.166419,77.667397


### N_connect_1_any

In [20]:
cores = np.array(['_num', '_size', ''])
filenames = '../data/N_connect_daily/N_connect_1_any/neutral_N_connect' + cores + '.feather'
factor_cols = ['neutral_factor'] * len(cores)
factor_names = 'N_connect' + cores
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    dirname='../data/N_connect_daily/N_connect_1_any/backtest1/'
)

print('-' * 10 + ' hs800 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs800',
    dirname='../data/N_connect_daily/N_connect_1_any/backtest1_hs800/'
)

print('-' * 10 + ' hs1000 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs1000',
    dirname='../data/N_connect_daily/N_connect_1_any/backtest1_hs1000/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,-0.015686,0.574581,-0.199529,-7.232747,0.225469,-0.252475
0,N_connect_size,-0.013074,0.569254,-0.174225,-6.315508,0.221153,-0.222306
0,N_connect,-0.014674,0.573820,-0.187020,-6.779318,0.229342,-0.247214


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.599298,0.119174,0.211959,-0.285459,0.562248,-0.417482,69.931517
0,N_connect_num,excess,0.473066,0.085605,0.125130,-0.249426,0.684129,-0.343209,69.931517
0,N_connect_num,long_short,0.627333,0.101161,0.077237,-0.090515,1.309754,-1.117612,69.931517
0,N_connect_size,long,0.464575,0.099598,0.208295,-0.300207,0.478157,-0.331763,30.714357
0,N_connect_size,excess,0.341990,0.066614,0.126875,-0.277778,0.525037,-0.239810,30.714357
0,N_connect_size,long_short,0.308479,0.056099,0.077727,-0.129288,0.721745,-0.433907,30.714357
0,N_connect,long,0.542354,0.111082,0.210544,-0.292872,0.527596,-0.379285,46.150522
0,N_connect,excess,0.417118,0.077755,0.126454,-0.251729,0.614888,-0.308885,46.150522
0,N_connect,long_short,0.487923,0.082599,0.079463,-0.100850,1.039467,-0.819025,46.150522


---------- hs800 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,-0.009277,0.557839,-0.122276,-4.432390,0.222734,-0.246418
0,N_connect_size,-0.007735,0.557078,-0.100642,-3.648201,0.256992,-0.239451
0,N_connect,-0.008813,0.557078,-0.113220,-4.104110,0.228029,-0.242284


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.478056,0.096100,0.183393,-0.286617,0.524013,-0.335291,72.619620
0,N_connect_num,excess,0.357596,0.063221,0.073121,-0.149084,0.864614,-0.424065,72.619620
0,N_connect_num,long_short,0.228978,0.044911,0.093720,-0.132095,0.479207,-0.339992,72.619620
0,N_connect_size,long,0.428016,0.087399,0.175842,-0.274943,0.497034,-0.317882,30.105063
0,N_connect_size,excess,0.299409,0.054780,0.078787,-0.205090,0.695298,-0.267103,30.105063
0,N_connect_size,long_short,0.040464,0.011695,0.089655,-0.142370,0.130441,-0.082143,30.105063
0,N_connect,long,0.425742,0.087787,0.179501,-0.284555,0.489058,-0.308505,46.636140
0,N_connect,excess,0.304060,0.055156,0.074489,-0.155383,0.740457,-0.354968,46.636140
0,N_connect,long_short,0.124358,0.027159,0.092932,-0.158449,0.292242,-0.171402,46.636140


---------- hs1000 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,N_connect_num,-0.015912,0.580670,-0.202528,-7.341459,0.232549,-0.243080
0,N_connect_size,-0.012907,0.563166,-0.174232,-6.315746,0.212887,-0.235373
0,N_connect,-0.014755,0.566971,-0.190444,-6.903434,0.217949,-0.244174


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,N_connect_num,long,0.600175,0.121128,0.219866,-0.302444,0.550916,-0.400496,75.167229
0,N_connect_num,excess,0.493924,0.087501,0.117376,-0.239046,0.745473,-0.366042,75.167229
0,N_connect_num,long_short,0.663597,0.106910,0.089079,-0.130699,1.200174,-0.817989,75.167229
0,N_connect_size,long,0.422307,0.094884,0.214825,-0.328025,0.441680,-0.289258,33.095502
0,N_connect_size,excess,0.320046,0.062041,0.117674,-0.246824,0.527229,-0.251358,33.095502
0,N_connect_size,long_short,0.371048,0.066445,0.087370,-0.163985,0.760499,-0.405189,33.095502
0,N_connect,long,0.515134,0.108781,0.217125,-0.304568,0.501009,-0.357166,49.933430
0,N_connect,excess,0.410264,0.075524,0.117153,-0.249742,0.644660,-0.302406,49.933430
0,N_connect,long_short,0.494171,0.084380,0.089475,-0.153243,0.943054,-0.550627,49.933430


## 跳跃关联动量因子回测

In [21]:
cores = np.array([
    'without_posjump_num', 'without_posjump_size',
    'without_posjump', 'without_posjump_residual'
])
filenames = '../data/peer_ret_daily/neutral_peer_' + cores + '.feather'
factor_names = 'relative_' + cores
factor_cols = ['neutral_factor'] * len(cores)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    dirname='../data/peer_ret_daily//backtest1/'
)

print('-' * 10 + ' hs800 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs800',
    dirname='../data/peer_ret_daily/backtest1_hs800/'
)

print('-' * 10 + ' hs1000 ' + '-' * 10)
set_backtest(
    filenames=filenames,
    factor_cols=factor_cols, factor_names=factor_names,
    benchmark='../data/benchmark/benchmark_000906_daily.feather',
    sample='hs1000',
    dirname='../data/peer_ret_daily/backtest1_hs1000/'
)

,factor,IC_mean,winning,IC_IR,t,max,min
0,relative_without_posjump_num,0.029492,0.665145,0.418545,15.171888,0.242724,-0.216659
0,relative_without_posjump_size,0.028909,0.668950,0.419637,15.211482,0.237605,-0.208667
0,relative_without_posjump,0.030796,0.662861,0.424084,15.372688,0.247819,-0.212322
0,relative_without_posjump_residual,0.008007,0.602740,0.189731,6.877576,0.145867,-0.168965


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,relative_without_posjump_num,long,1.187595,0.187697,0.208905,-0.257442,0.898478,-0.729084,85.686823
0,relative_without_posjump_num,excess,1.003632,0.152082,0.128670,-0.253955,1.181950,-0.598853,85.686823
0,relative_without_posjump_num,long_short,2.772650,0.295039,0.088465,-0.066488,3.335078,-4.437475,85.686823
0,relative_without_posjump_size,long,1.145771,0.184169,0.212403,-0.264620,0.867077,-0.695978,74.276499
0,relative_without_posjump_size,excess,0.972929,0.148659,0.128534,-0.258983,1.156576,-0.574012,74.276499
0,relative_without_posjump_size,long_short,2.855283,0.300411,0.088274,-0.079850,3.403164,-3.762178,74.276499
0,relative_without_posjump,long,1.238976,0.193051,0.209148,-0.256657,0.923034,-0.752174,76.218206
0,relative_without_posjump,excess,1.051536,0.157275,0.128416,-0.264036,1.224733,-0.595659,76.218206
0,relative_without_posjump,long_short,3.000127,0.310095,0.092133,-0.077678,3.365719,-3.992062,76.218206
0,relative_without_posjump_residual,long,0.695097,0.135553,0.227181,-0.306386,0.596671,-0.442424,94.095660


---------- hs800 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,relative_without_posjump_num,0.019941,0.594368,0.215475,7.810776,0.348264,-0.325717
0,relative_without_posjump_size,0.018952,0.589802,0.211103,7.652292,0.342966,-0.291756
0,relative_without_posjump,0.020460,0.601979,0.217300,7.876949,0.357060,-0.319474
0,relative_without_posjump_residual,0.006786,0.560122,0.107472,3.895772,0.205488,-0.189034


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,relative_without_posjump_num,long,0.614201,0.114768,0.183458,-0.210587,0.625578,-0.544988,78.842034
0,relative_without_posjump_num,excess,0.465388,0.081331,0.099227,-0.242122,0.819645,-0.335909,78.842034
0,relative_without_posjump_num,long_short,0.963569,0.145229,0.111554,-0.119980,1.301869,-1.210445,78.842034
0,relative_without_posjump_size,long,0.628071,0.117278,0.186779,-0.233254,0.627895,-0.502789,71.787255
0,relative_without_posjump_size,excess,0.486338,0.083766,0.094301,-0.229550,0.888277,-0.364913,71.787255
0,relative_without_posjump_size,long_short,0.949675,0.143315,0.108694,-0.110524,1.318513,-1.296687,71.787255
0,relative_without_posjump,long,0.677780,0.123559,0.185878,-0.211846,0.664733,-0.583251,72.685974
0,relative_without_posjump,excess,0.526706,0.089860,0.099184,-0.244304,0.905995,-0.367821,72.685974
0,relative_without_posjump,long_short,0.995112,0.149032,0.113871,-0.110606,1.308781,-1.347415,72.685974
0,relative_without_posjump_residual,long,0.370989,0.083203,0.197043,-0.284821,0.422256,-0.292123,91.325745


---------- hs1000 ----------


,factor,IC_mean,winning,IC_IR,t,max,min
0,relative_without_posjump_num,0.027198,0.639269,0.337370,12.229354,0.284446,-0.251597
0,relative_without_posjump_size,0.027360,0.643075,0.348262,12.624204,0.287595,-0.233883
0,relative_without_posjump,0.028755,0.641553,0.349660,12.674877,0.294976,-0.248199
0,relative_without_posjump_residual,0.008596,0.576104,0.150072,5.439972,0.175711,-0.200401


,factor_name,group_name,tot,annual,volatility,max_drawdown,sharpe,calmar,turnover_annual
0,relative_without_posjump_num,long,1.155659,0.185970,0.215549,-0.281409,0.862770,-0.660850,86.745525
0,relative_without_posjump_num,excess,0.991541,0.150406,0.126471,-0.252962,1.189251,-0.594578,86.745525
0,relative_without_posjump_num,long_short,2.657987,0.289352,0.104273,-0.080446,2.774958,-3.596865,86.745525
0,relative_without_posjump_size,long,0.840485,0.151429,0.219022,-0.302681,0.691385,-0.500292,74.385874
0,relative_without_posjump_size,excess,0.709025,0.116897,0.124625,-0.248937,0.937983,-0.469584,74.385874
0,relative_without_posjump_size,long_short,2.389795,0.270010,0.099191,-0.077703,2.722138,-3.474899,74.385874
0,relative_without_posjump,long,1.024975,0.171865,0.215610,-0.275400,0.797110,-0.624057,77.603283
0,relative_without_posjump,excess,0.873015,0.136723,0.124832,-0.257625,1.095252,-0.530703,77.603283
0,relative_without_posjump,long_short,2.728020,0.294126,0.104830,-0.072377,2.805747,-4.063799,77.603283
0,relative_without_posjump_residual,long,0.379201,0.093623,0.235694,-0.355768,0.397222,-0.263156,95.752294
